In [3]:
import os, shutil, subprocess

# Adjust this if your site uses a slightly different path:
JAVA_HOME = "/rds/bear-apps/2024a/EL8-ice/software/Java/17.0.15"

os.environ["JAVA_HOME"] = JAVA_HOME
os.environ["PATH"] = f"{JAVA_HOME}/bin:" + os.environ["PATH"]

print("which java ->", shutil.which("java"))
print(subprocess.check_output(["java","-version"], stderr=subprocess.STDOUT).decode())

import h2o
import os
nthreads = int(os.getenv("SLURM_CPUS_PER_TASK","-1"))
h2o.init(nthreads=nthreads, max_mem_size="120G", port=54321, bind_to_localhost=True)


which java -> /rds/bear-apps/2024a/EL8-ice/software/Java/17.0.15/bin/java
openjdk version "17.0.15" 2025-04-15
OpenJDK Runtime Environment Temurin-17.0.15+6 (build 17.0.15+6)
OpenJDK 64-Bit Server VM Temurin-17.0.15+6 (build 17.0.15+6, mixed mode, sharing)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17.0.15" 2025-04-15; OpenJDK Runtime Environment Temurin-17.0.15+6 (build 17.0.15+6); OpenJDK 64-Bit Server VM Temurin-17.0.15+6 (build 17.0.15+6, mixed mode, sharing)
  Starting server from /rds/homes/j/jxq370/.local/lib/python3.12/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp93yiia1c
  JVM stdout: /tmp/tmp93yiia1c/h2o_jxq370_started_from_python.out
  JVM stderr: /tmp/tmp93yiia1c/h2o_jxq370_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install th

H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/London
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,5 months and 13 days
H2O_cluster_name:,H2O_from_python_jxq370_rvkv9f
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,120 Gb
H2O_cluster_total_cores:,72
H2O_cluster_allowed_cores:,1
H2O_cluster_status:,"locked, healthy"


In [4]:
#!/usr/bin/env python3
"""
EPC Step 3: Scenario Generation and Impact Prediction 
======================================================================

This script generates upgrade scenarios for sub-standard homes using:
1. H2O AutoML model from Step 1 to predict EPC rating improvements
2. Clustering results from Step 2 to identify upgrade pathways
3. Cost-benefit analysis of proposed upgrades
4. Analysis of upgrade costs by initial rating band (D→C, E→C, F→C, G→C)

CRITICAL FIXES APPLIED:
- Fixed upgrade persistence (using .loc instead of .iloc)
- Fixed glazing proportion scaling (percentage to ratio conversion)
- Ensured A1 > A0 with sensible minimum targets
- Linked description scores to actual efficiency changes
- Added data diagnostics
- Realistic cost calculations
- Added rating band cost analysis

Author: [Your Name]
Date: [Current Date]
"""

import os
import re
import glob
import json
import logging
import warnings
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass, field

import numpy as np
import pandas as pd
import h2o
from h2o import H2OFrame

warnings.filterwarnings("ignore")

# --------------------------- CONFIG ------------------------------------------

@dataclass
class Config:
    """Configuration for Step 3 scenario generation."""
    
    # Step-2 outputs directory
    step2_dir: str = "~/shiz-wm-netzero/users/juncheng/EPC/outputs/step_2_clustering"
    
    # Step-1 outputs directory (for feature importance)
    step1_dir: str = "~/shiz-wm-netzero/users/juncheng/EPC/automl/outputs/base_model_importance"
    
    # Step-1 H2O model directory
    h2o_model_dir: str = "~/shiz-wm-netzero/users/juncheng/EPC/automl/outputs/epc_automl_model"
    
    # Step-3 outputs directory
    output_dir: str = "~/shiz-wm-netzero/users/juncheng/EPC/outputs/step_3_scenarios"
    
    # H2O settings
    h2o_port: str = "54321"
    h2o_memory: str = "256G"
    
    # Scenario parameters
    a0_percentile: int = 60  # Moderate upgrades (60th percentile of C-rated homes)
    a1_percentile: int = 85  # Ambitious upgrades (85th percentile of C-rated homes)
    
    # Cost parameters (£ per unit)
    costs: Dict[str, float] = field(default_factory=lambda: {
        'wall_insulation_cavity': 15.0,  # per sqm
        'wall_insulation_solid': 100.0,  # per sqm
        'roof_insulation': 25.0,  # per sqm
        'floor_insulation': 40.0,  # per sqm
        'double_glazing': 350.0,  # per sqm
        'heating_system_upgrade': 3500.0,  # per dwelling
        'hot_water_upgrade': 1500.0,  # per dwelling
        'lighting_upgrade': 200.0,  # per dwelling
    })
    
    # Energy price assumptions
    energy_price_per_kwh: float = 0.28  # £/kWh (2024 prices)
    
    # All upgradeable features (not dependent on importance)
    numeric_fabric_features: List[str] = field(default_factory=lambda: [
        "WALLS_ENERGY_EFF", "ROOF_ENERGY_EFF", "WINDOWS_ENERGY_EFF", "FLOOR_ENERGY_EFF"
    ])
    
    service_features: List[str] = field(default_factory=lambda: [
        "MAINHEAT_ENERGY_EFF", "HOT_WATER_ENERGY_EFF", "LIGHTING_ENERGY_EFF", "MAINHEATC_ENERGY_EFF"
    ])
    
    categorical_fabric_features: List[str] = field(default_factory=lambda: [
        "ROOF_DESCRIPTION", "WALLS_DESCRIPTION", "FLOOR_DESCRIPTION"
    ])
    
    numerical_features: List[str] = field(default_factory=lambda: [
        "TOTAL_FLOOR_AREA", "LOW_ENERGY_LIGHTING", "FLOOR_HEIGHT",
        "PHOTO_SUPPLY", "MULTI_GLAZE_PROPORTION"
    ])
    
    categorical_features: List[str] = field(default_factory=lambda: [
        "PROPERTY_TYPE", "ENERGY_TARIFF", "FLAT_TOP_STOREY",
        "HEAT_LOSS_CORRIDOR", "MAIN_FUEL", "CONSTRUCTION_AGE_BAND",
        "TENURE", "SECONDHEAT_DESCRIPTION"
    ])


class ScenarioGenerator:
    """Main class for scenario generation and impact prediction."""
    
    def __init__(self, config: Config):
        """Initialize the scenario generator."""
        self.config = config
        self.setup_logging()
        self.setup_directories()
        
        # Data containers
        self.epc_c_data: Optional[pd.DataFrame] = None
        self.substandard_data: Optional[pd.DataFrame] = None
        self.h2o_model = None
        self.feature_importance: Optional[pd.DataFrame] = None
        self.cluster_profiles: Optional[pd.DataFrame] = None
        
        # Track available features
        self.available_numeric_fabric: List[str] = []
        self.available_service: List[str] = []
        self.available_categorical_fabric: List[str] = []
        
        # Results containers
        self.baseline_df: Optional[pd.DataFrame] = None
        self.a0_scenario_df: Optional[pd.DataFrame] = None
        self.a1_scenario_df: Optional[pd.DataFrame] = None
        self.cost_benefit_df: Optional[pd.DataFrame] = None
        self.rating_cost_df: Optional[pd.DataFrame] = None
    
    def setup_logging(self) -> None:
        """Setup logging configuration."""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.run_timestamp = timestamp
        
        # Create output directory for this run
        self.run_output_dir = Path(self.config.output_dir).expanduser() / f"scenarios_{timestamp}"
        self.run_output_dir.mkdir(parents=True, exist_ok=True)
        
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(self.run_output_dir / 'scenario_generation.log'),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)
    
    def setup_directories(self) -> None:
        """Create output directories."""
        self.plots_dir = self.run_output_dir / "plots"
        self.results_dir = self.run_output_dir / "results"
        
        for directory in [self.plots_dir, self.results_dir]:
            directory.mkdir(parents=True, exist_ok=True)
    
    def init_h2o(self) -> None:
        """Initialize H2O and load the AutoML model."""
        try:
            # Initialize H2O
            h2o.init(
                port=self.config.h2o_port,
                max_mem_size=self.config.h2o_memory,
                nthreads=-1
            )
            self.logger.info("H2O initialized successfully")
            
            # Find and load the saved model
            model_dir = Path(self.config.h2o_model_dir).expanduser()
            model_files = list(model_dir.glob("*"))
            
            if not model_files:
                raise FileNotFoundError(f"No model found in {model_dir}")
            
            # Load the first model found (should be the best model from Step 1)
            model_path = str(model_files[0])
            self.h2o_model = h2o.load_model(model_path)
            self.logger.info(f"Loaded H2O model: {self.h2o_model.model_id}")
            
            # Load feature importance from saved CSV file
            self._load_feature_importance_from_csv()
            
        except Exception as e:
            self.logger.error(f"Failed to initialize H2O or load model: {e}")
            raise
    
    def _load_feature_importance_from_csv(self) -> None:
        """Load feature importance from the saved CSV file from Step 1."""
        try:
            # Try multiple possible locations for the feature importance file
            possible_paths = [
                Path(self.config.step1_dir).expanduser() / 'combined_base_model_importance.csv',
                Path(self.config.step1_dir).expanduser() / 'feature_importance.csv',
                Path("~/shiz-wm-netzero/users/juncheng/EPC/automl/outputs/base_model_importance").expanduser() / 'combined_base_model_importance.csv'
            ]
            
            importance_file = None
            for path in possible_paths:
                if path.exists():
                    importance_file = path
                    break
            
            if importance_file is None:
                self.logger.warning("Feature importance file not found in expected locations")
                self.logger.info("Will proceed without feature importance (all features will be treated equally)")
                return
            
            self.logger.info(f"Loading feature importance from: {importance_file}")
            
            # Load the feature importance
            self.feature_importance = pd.read_csv(importance_file)
            
            # Check if the expected columns exist
            if 'feature' in self.feature_importance.columns and 'mean_importance' in self.feature_importance.columns:
                # Ensure that the importance columns are numeric
                self.feature_importance['mean_importance'] = pd.to_numeric(
                    self.feature_importance['mean_importance'], errors='coerce'
                )
                
                # Sort the data by mean importance in descending order
                self.feature_importance = self.feature_importance.sort_values(
                    by='mean_importance', ascending=False
                )
                
                self.logger.info(f"Loaded importance for {len(self.feature_importance)} features")
                
                # Log top 10 features
                self.logger.info("Top 10 most important features:")
                for i, row in self.feature_importance.head(10).iterrows():
                    self.logger.info(f"  {row['feature']}: {row['mean_importance']:.2f}")
            else:
                self.logger.warning("Feature importance file missing required columns")
                self.feature_importance = None
            
        except Exception as e:
            self.logger.warning(f"Could not load feature importance from CSV: {e}")
            self.logger.info("Will proceed without feature importance (all features will be treated equally)")
            self.feature_importance = None
    
    def load_step2_data(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Load the latest clustering results from Step 2."""
        step2_dir = Path(self.config.step2_dir).expanduser()
        
        # Find the most recent clustering run
        run_dirs = sorted(step2_dir.glob("clustering_*"))
        if not run_dirs:
            raise FileNotFoundError(f"No clustering results found in {step2_dir}")
        
        latest_dir = run_dirs[-1]
        results_dir = latest_dir / "results"
        
        self.logger.info(f"Loading Step 2 data from: {latest_dir.name}")
        
        # Load EPC C clustered data
        epc_c_files = list(results_dir.glob("epc_c_clustered_*.csv"))
        if not epc_c_files:
            raise FileNotFoundError("No EPC C clustered data found")
        
        epc_c_data = pd.read_csv(epc_c_files[-1])
        self.logger.info(f"Loaded {len(epc_c_data)} EPC C homes")
        
        # Load substandard matched data
        substandard_files = list(results_dir.glob("substandard_matched_*.csv"))
        if not substandard_files:
            raise FileNotFoundError("No substandard matched data found")
        
        substandard_data = pd.read_csv(substandard_files[-1])
        self.logger.info(f"Loaded {len(substandard_data)} substandard homes")
        
        # Identify available upgradeable features
        self._identify_available_features(epc_c_data)
        
        # Load cluster profiles if available
        profile_files = list(results_dir.glob("cluster_profiles_*.csv"))
        if profile_files:
            self.cluster_profiles = pd.read_csv(profile_files[-1])
            self.logger.info(f"Loaded profiles for {len(self.cluster_profiles)} clusters")
        
        return epc_c_data, substandard_data
    
    def _identify_available_features(self, df: pd.DataFrame) -> None:
        """Identify which upgradeable features are available in the data."""
        # Check numeric fabric features
        self.available_numeric_fabric = [
            f for f in self.config.numeric_fabric_features 
            if f in df.columns
        ]
        self.logger.info(f"Available numeric fabric features: {self.available_numeric_fabric}")
        
        # Check service features
        self.available_service = [
            f for f in self.config.service_features 
            if f in df.columns
        ]
        self.logger.info(f"Available service features: {self.available_service}")
        
        # Check categorical fabric features
        self.available_categorical_fabric = [
            f for f in self.config.categorical_fabric_features 
            if f in df.columns
        ]
        self.logger.info(f"Available categorical fabric features: {self.available_categorical_fabric}")
    
    def diagnose_data(self):
        """Run diagnostics on the data to catch issues early."""
        self.logger.info("\n" + "="*50)
        self.logger.info("DATA DIAGNOSTICS")
        self.logger.info("="*50)
        
        # Check floor area distribution
        floor_areas = pd.to_numeric(self.substandard_data['TOTAL_FLOOR_AREA'], errors='coerce')
        self.logger.info(f"Floor area stats:")
        self.logger.info(f"  Mean: {floor_areas.mean():.2f}")
        self.logger.info(f"  Median: {floor_areas.median():.2f}")
        self.logger.info(f"  95th percentile: {floor_areas.quantile(0.95):.2f}")
        self.logger.info(f"  99th percentile: {floor_areas.quantile(0.99):.2f}")
        self.logger.info(f"  Max: {floor_areas.max():.2f}")
        
        if floor_areas.mean() > 500:
            self.logger.warning("  WARNING: Floor areas may be in square feet, not square meters!")
        
        # Check glazing proportion
        if 'MULTI_GLAZE_PROPORTION' in self.substandard_data.columns:
            glazing = pd.to_numeric(self.substandard_data['MULTI_GLAZE_PROPORTION'], errors='coerce')
            if len(glazing.dropna()) > 0:
                self.logger.info(f"Glazing proportion stats:")
                self.logger.info(f"  Mean: {glazing.mean():.2f}")
                self.logger.info(f"  Median: {glazing.median():.2f}")
                self.logger.info(f"  Max: {glazing.max():.2f}")
                if glazing.mean() > 1:
                    self.logger.warning("  WARNING: Glazing appears to be percentage (0-100), not ratio (0-1)!")
        
        # Check current efficiency distributions
        self.logger.info("\nCurrent efficiency distributions:")
        for feature in self.available_numeric_fabric[:3]:
            numeric_vals = self._convert_efficiency_to_numeric(self.substandard_data[feature])
            valid_vals = numeric_vals.dropna()
            if len(valid_vals) > 0:
                self.logger.info(f"{feature}:")
                self.logger.info(f"  Mean: {valid_vals.mean():.2f}")
                self.logger.info(f"  % below Good (4): {(valid_vals < 4).mean()*100:.1f}%")
                self.logger.info(f"  % at Very Poor (1): {(valid_vals == 1).mean()*100:.1f}%")
    
    # Improved description scoring functions
    
    def _has_negative_insulation(self, text: str) -> bool:
        """Check if text contains negative insulation mentions."""
        negative_patterns = [
            "no insulation", "not insulated", "uninsulated", 
            "without insulation", "no loft insulation"
        ]
        return any(pattern in text for pattern in negative_patterns)
    
    def _extract_uvalue(self, text: str) -> Optional[float]:
        """Extract U-value from text (tolerant of encoding issues)."""
        # Match patterns like "0.25 W/m²K" or "0.25 w/m-2k"
        match = re.search(r'(\d+(?:\.\d+)?)\s*w/m', text)
        if match:
            try:
                return float(match.group(1))
            except ValueError:
                return None
        return None
    
    def score_roof_desc(self, txt: Any) -> float:
        """Score roof description (0-2 scale) with robust parsing."""
        if pd.isna(txt) or not isinstance(txt, str):
            return 0.0
        
        t = txt.lower()
        
        # Check for negative mentions first
        if self._has_negative_insulation(t):
            return 0.0
        
        # Check for partial/limited insulation
        if "partial" in t or "limited" in t:
            return 0.5
        
        # Check U-values (roof thresholds)
        u_value = self._extract_uvalue(t)
        if u_value is not None:
            if u_value <= 0.12:
                return 2.0  # Excellent
            elif u_value <= 0.16:
                return 1.5  # Good
            elif u_value <= 0.25:
                return 1.0  # Moderate
            else:
                return 0.5  # Poor but present
        
        # Check thickness patterns
        thickness_match = re.search(r'\b(\d{2,3})\s*mm\b', t)
        if thickness_match:
            thickness = int(thickness_match.group(1))
            if thickness >= 250:
                return 2.0
            elif thickness >= 150:
                return 1.5
            elif thickness >= 75:
                return 1.0
            else:
                return 0.5
        
        # Generic insulation mention
        if "insulat" in t:
            return 1.0
        
        return 0.0
    
    def score_walls_desc(self, txt: Any) -> float:
        """Score walls description (0-2 scale) with robust parsing."""
        if pd.isna(txt) or not isinstance(txt, str):
            return 0.0
        
        t = txt.lower()
        
        # Check for negative mentions first
        if self._has_negative_insulation(t):
            return 0.0
        
        # Check for partial/limited insulation
        if "partial" in t or "limited" in t:
            return 0.5
        
        # Check for external/internal wall insulation
        if any(k in t for k in ["external insulation", "internal insulation", " ewi", " iwi"]):
            return 2.0
        
        # Check U-values (wall thresholds)
        u_value = self._extract_uvalue(t)
        if u_value is not None:
            if u_value <= 0.18:
                return 2.0  # Excellent
            elif u_value <= 0.30:
                return 1.5  # Good
            elif u_value <= 0.45:
                return 1.0  # Moderate
            else:
                return 0.0  # Too high
        
        # Check for cavity wall insulation
        if "cavity" in t and ("filled" in t or "insulat" in t):
            return 2.0
        
        # Check for solid wall with insulation
        if "solid" in t and "insulat" in t:
            return 2.0
        
        # Generic insulation mention
        if "insulat" in t:
            return 1.0
        
        # Cavity wall without insulation
        if "cavity" in t and "unfilled" not in t:
            return 0.3  # Has potential for improvement
        
        return 0.0
    
    def score_floor_desc(self, txt: Any) -> float:
        """Score floor description (0-1 scale) with robust parsing."""
        if pd.isna(txt) or not isinstance(txt, str):
            return 0.0
        
        t = txt.lower()
        
        # Check for negative mentions first
        if self._has_negative_insulation(t):
            return 0.0
        
        # Check for partial/limited insulation
        if "partial" in t or "limited" in t:
            return 0.5
        
        # Check U-values (floor thresholds)
        u_value = self._extract_uvalue(t)
        if u_value is not None:
            if u_value <= 0.18:
                return 1.0  # Excellent
            elif u_value <= 0.25:
                return 0.8  # Good
            elif u_value <= 0.45:
                return 0.5  # Moderate
            else:
                return 0.0  # Too high
        
        # Check for insulation
        if "insulat" in t:
            return 1.0
        
        # Suspended timber floor (has upgrade potential)
        if "suspended" in t and "timber" in t:
            return 0.3
        
        return 0.0
    
    def add_desc_scores(self, df: pd.DataFrame) -> pd.DataFrame:
        """Add description scores for available categorical features."""
        df = df.copy()
        
        # Always add scores when columns exist (not dependent on importance)
        if "ROOF_DESCRIPTION" in df.columns:
            df["ROOF_DESC_SCORE"] = df["ROOF_DESCRIPTION"].apply(self.score_roof_desc)
            self.logger.info(f"  Added ROOF_DESC_SCORE (mean: {df['ROOF_DESC_SCORE'].mean():.2f})")
        
        if "WALLS_DESCRIPTION" in df.columns:
            df["WALLS_DESC_SCORE"] = df["WALLS_DESCRIPTION"].apply(self.score_walls_desc)
            self.logger.info(f"  Added WALLS_DESC_SCORE (mean: {df['WALLS_DESC_SCORE'].mean():.2f})")
        
        if "FLOOR_DESCRIPTION" in df.columns:
            df["FLOOR_DESC_SCORE"] = df["FLOOR_DESCRIPTION"].apply(self.score_floor_desc)
            self.logger.info(f"  Added FLOOR_DESC_SCORE (mean: {df['FLOOR_DESC_SCORE'].mean():.2f})")
        
        return df
    
    def _convert_efficiency_to_numeric(self, series: pd.Series) -> pd.Series:
        """Convert efficiency ratings to numeric scale, treating Unknown/N/A as NaN."""
        mapping = {
            "Very Good": 5,
            "Good": 4,
            "Average": 3,
            "Poor": 2,
            "Very Poor": 1,
            "N/A": np.nan,
            "Unknown": np.nan,
            "": np.nan
        }
        
        if series.dtype == 'object' or pd.api.types.is_categorical_dtype(series):
            # Don't fill NaN with Average - keep as NaN
            return series.map(mapping)
        else:
            return series
    
    def _numeric_to_efficiency(self, value: float) -> str:
        """Convert numeric efficiency back to categorical with proper clamping."""
        if pd.isna(value):
            return "Unknown"
        
        # Ensure value is in valid range [1, 5]
        value = np.clip(value, 1, 5)
        
        # Round to nearest integer for mapping
        value = round(value)
        
        mapping = {
            5: "Very Good",
            4: "Good",
            3: "Average",
            2: "Poor",
            1: "Very Poor"
        }
        
        return mapping.get(value, "Average")
    
    def compute_upgrade_targets(self, epc_c_data: pd.DataFrame) -> Dict[int, Dict[str, Dict[str, float]]]:
        """Compute upgrade targets based on EPC C home characteristics."""
        # Add description scores first
        epc_c_data = self.add_desc_scores(epc_c_data)
        
        targets = {}
        clusters = sorted(epc_c_data['cluster'].unique())
        
        for cluster_id in clusters:
            cluster_data = epc_c_data[epc_c_data['cluster'] == cluster_id].copy()
            
            targets[cluster_id] = {
                'A0': {},  # Moderate upgrades
                'A1': {}   # Ambitious upgrades
            }
            
            # Process numeric fabric features (always include if available)
            for feature in self.available_numeric_fabric:
                numeric_vals = self._convert_efficiency_to_numeric(cluster_data[feature])
                numeric_vals = numeric_vals.dropna()
                
                if len(numeric_vals) > 0:
                    # Calculate percentiles and ensure valid range
                    a0_val = np.percentile(numeric_vals, self.config.a0_percentile)
                    a1_val = np.percentile(numeric_vals, self.config.a1_percentile)
                    
                    # Clamp to valid range [1, 5] and round
                    targets[cluster_id]['A0'][feature] = np.clip(np.ceil(a0_val), 1, 5)
                    targets[cluster_id]['A1'][feature] = np.clip(np.ceil(a1_val), 1, 5)
            
            # Process service features (always include if available)
            for feature in self.available_service:
                numeric_vals = self._convert_efficiency_to_numeric(cluster_data[feature])
                numeric_vals = numeric_vals.dropna()
                
                if len(numeric_vals) > 0:
                    a0_val = np.percentile(numeric_vals, self.config.a0_percentile)
                    a1_val = np.percentile(numeric_vals, self.config.a1_percentile)
                    
                    targets[cluster_id]['A0'][feature] = np.clip(np.ceil(a0_val), 1, 5)
                    targets[cluster_id]['A1'][feature] = np.clip(np.ceil(a1_val), 1, 5)
            
            # Process description scores
            for score_col in ['ROOF_DESC_SCORE', 'WALLS_DESC_SCORE', 'FLOOR_DESC_SCORE']:
                if score_col in cluster_data.columns:
                    score_vals = cluster_data[score_col].dropna()
                    if len(score_vals) > 0:
                        targets[cluster_id]['A0'][score_col] = np.percentile(
                            score_vals, self.config.a0_percentile
                        )
                        targets[cluster_id]['A1'][score_col] = np.percentile(
                            score_vals, self.config.a1_percentile
                        )
            
            # Process upgradeable numerical features
            upgradeable_numerical = ['LOW_ENERGY_LIGHTING', 'PHOTO_SUPPLY']
            for feature in upgradeable_numerical:
                if feature in cluster_data.columns:
                    numeric_vals = pd.to_numeric(cluster_data[feature], errors='coerce').dropna()
                    if len(numeric_vals) > 0:
                        targets[cluster_id]['A0'][feature] = np.percentile(
                            numeric_vals, self.config.a0_percentile
                        )
                        targets[cluster_id]['A1'][feature] = np.percentile(
                            numeric_vals, self.config.a1_percentile
                        )
        
        # CRITICAL FIX: Ensure A1 >= A0 and apply sensible minimum targets
        for cluster_id in targets:
            for feature in targets[cluster_id]['A0']:
                if feature in targets[cluster_id]['A1']:
                    # A1 must be at least as good as A0
                    targets[cluster_id]['A1'][feature] = max(
                        targets[cluster_id]['A1'][feature],
                        targets[cluster_id]['A0'][feature]
                    )
                    
                    # Special fix for windows - never target "Very Poor" for C-rated homes
                    if feature == 'WINDOWS_ENERGY_EFF':
                        targets[cluster_id]['A0'][feature] = max(3, targets[cluster_id]['A0'][feature])  # At least Average
                        targets[cluster_id]['A1'][feature] = max(4, targets[cluster_id]['A1'][feature])  # At least Good
                    
                    # Ensure fabric features target at least "Average" for A0, "Good" for A1
                    if feature in ['WALLS_ENERGY_EFF', 'ROOF_ENERGY_EFF', 'FLOOR_ENERGY_EFF']:
                        targets[cluster_id]['A0'][feature] = max(3, targets[cluster_id]['A0'][feature])
                        targets[cluster_id]['A1'][feature] = max(4, targets[cluster_id]['A1'][feature])
                    
                    # Ensure heating/hot water target at least "Average" for A0, "Good" for A1
                    if feature in ['MAINHEAT_ENERGY_EFF', 'HOT_WATER_ENERGY_EFF']:
                        targets[cluster_id]['A0'][feature] = max(3, targets[cluster_id]['A0'][feature])
                        targets[cluster_id]['A1'][feature] = max(4, targets[cluster_id]['A1'][feature])
        
        self.logger.info(f"Computed upgrade targets for {len(targets)} clusters")
        self._log_target_summary(targets)
        
        return targets
    
    def _log_target_summary(self, targets: Dict) -> None:
        """Log summary of computed targets."""
        for cluster_id in targets:
            self.logger.info(f"\nCluster {cluster_id} targets:")
            for scenario in ['A0', 'A1']:
                self.logger.info(f"  {scenario}:")
                for feature, value in targets[cluster_id][scenario].items():
                    if feature in self.available_numeric_fabric + self.available_service:
                        # Convert back to label for logging
                        label = self._numeric_to_efficiency(value)
                        self.logger.info(f"    {feature}: {value} ({label})")
                    else:
                        self.logger.info(f"    {feature}: {value:.2f}")
    
    def create_upgrade_scenarios(self, substandard_data: pd.DataFrame, 
                                targets: Dict) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
        """Create A0 and A1 upgrade scenarios for substandard homes."""
        # Add description scores to substandard data
        substandard_data = self.add_desc_scores(substandard_data)
        
        # Create copies for scenarios
        baseline_df = substandard_data.copy()
        a0_df = substandard_data.copy()
        a1_df = substandard_data.copy()
        
        # Track upgrades applied
        a0_upgrades = []
        a1_upgrades = []
        
        for idx in range(len(substandard_data)):
            cluster_id = int(substandard_data.iloc[idx]['matched_cluster'])
            
            if cluster_id not in targets:
                self.logger.warning(f"No targets for cluster {cluster_id}")
                a0_upgrades.append({})
                a1_upgrades.append({})
                continue
            
            # CRITICAL FIX: Apply upgrades directly to DataFrame using .loc
            # Apply A0 (moderate) upgrades
            a0_home_upgrades = self._apply_upgrades_inplace(
                a0_df, idx, targets[cluster_id]['A0'], 'A0'
            )
            a0_upgrades.append(a0_home_upgrades)
            
            # Apply A1 (ambitious) upgrades  
            a1_home_upgrades = self._apply_upgrades_inplace(
                a1_df, idx, targets[cluster_id]['A1'], 'A1'
            )
            a1_upgrades.append(a1_home_upgrades)
        
        # Add upgrade tracking columns
        a0_df['upgrades_applied'] = a0_upgrades
        a1_df['upgrades_applied'] = a1_upgrades
        a0_df['scenario'] = 'A0_moderate'
        a1_df['scenario'] = 'A1_ambitious'
        baseline_df['scenario'] = 'baseline'
        
        # Log upgrade statistics
        self._log_upgrade_statistics(a0_upgrades, a1_upgrades)
        
        return baseline_df, a0_df, a1_df
    
    def _apply_upgrades_inplace(self, df: pd.DataFrame, idx: int, targets: Dict[str, float], 
                                scenario: str) -> Dict[str, bool]:
        """Apply upgrades directly to DataFrame row (in-place modification)."""
        upgrades = {}
        
        # Apply numeric fabric upgrades
        for feature in self.available_numeric_fabric:
            if feature in targets and feature in df.columns:
                current_val = self._convert_efficiency_to_numeric(
                    pd.Series([df.loc[idx, feature]])
                ).iloc[0]
                target_val = targets[feature]
                
                if not pd.isna(current_val) and not pd.isna(target_val):
                    if target_val > current_val:
                        # CRITICAL: Use .loc to modify in-place
                        df.loc[idx, feature] = self._numeric_to_efficiency(target_val)
                        upgrades[feature] = True
                    else:
                        upgrades[feature] = False
        
        # Apply service upgrades
        for feature in self.available_service:
            if feature in targets and feature in df.columns:
                current_val = self._convert_efficiency_to_numeric(
                    pd.Series([df.loc[idx, feature]])
                ).iloc[0]
                target_val = targets[feature]
                
                if not pd.isna(current_val) and not pd.isna(target_val):
                    if target_val > current_val:
                        df.loc[idx, feature] = self._numeric_to_efficiency(target_val)
                        upgrades[feature] = True
                    else:
                        upgrades[feature] = False
        
        # FIX: When description scores improve, ALSO update the efficiency ratings
        for score_col, desc_col, eff_col in [
            ('ROOF_DESC_SCORE', 'ROOF_DESCRIPTION', 'ROOF_ENERGY_EFF'),
            ('WALLS_DESC_SCORE', 'WALLS_DESCRIPTION', 'WALLS_ENERGY_EFF'),
            ('FLOOR_DESC_SCORE', 'FLOOR_DESCRIPTION', 'FLOOR_ENERGY_EFF')
        ]:
            if score_col in targets and score_col in df.columns and eff_col in df.columns:
                current_score = df.loc[idx, score_col] if score_col in df.columns else 0
                target_score = targets[score_col]
                
                if not pd.isna(current_score) and not pd.isna(target_score):
                    if target_score > current_score:
                        # Update the score
                        df.loc[idx, score_col] = target_score
                        
                        # CRITICAL: Also update the efficiency rating based on improved score
                        current_eff = self._convert_efficiency_to_numeric(
                            pd.Series([df.loc[idx, eff_col]])
                        ).iloc[0] if eff_col in df.columns else 1
                        
                        # Map improved score to improved efficiency
                        if scenario == 'A0' and target_score >= 1.0:
                            new_eff = max(4, current_eff if not pd.isna(current_eff) else 3)  # At least "Good"
                        elif scenario == 'A1' and target_score >= 1.5:
                            new_eff = 5  # "Very Good" for ambitious
                        else:
                            new_eff = max(3, current_eff if not pd.isna(current_eff) else 2)  # At least "Average"
                        
                        df.loc[idx, eff_col] = self._numeric_to_efficiency(new_eff)
                        upgrades[eff_col] = True
        
        # Apply numerical feature upgrades
        for feature in ['LOW_ENERGY_LIGHTING', 'PHOTO_SUPPLY']:
            if feature in targets and feature in df.columns:
                current_val = pd.to_numeric(df.loc[idx, feature], errors='coerce')
                target_val = targets[feature]
                
                if not pd.isna(current_val) and not pd.isna(target_val):
                    if target_val > current_val:
                        df.loc[idx, feature] = target_val
                        upgrades[feature] = True
        
        return upgrades
    
    def _log_upgrade_statistics(self, a0_upgrades: List[Dict], a1_upgrades: List[Dict]):
        """Log statistics about upgrades applied."""
        self.logger.info("\n" + "="*60)
        self.logger.info("UPGRADE APPLICATION STATISTICS")
        self.logger.info("="*60)
        
        # Count upgrades by feature
        a0_counts = {}
        a1_counts = {}
        
        for upgrades in a0_upgrades:
            for feature, upgraded in upgrades.items():
                if upgraded:
                    a0_counts[feature] = a0_counts.get(feature, 0) + 1
        
        for upgrades in a1_upgrades:
            for feature, upgraded in upgrades.items():
                if upgraded:
                    a1_counts[feature] = a1_counts.get(feature, 0) + 1
        
        self.logger.info("\nA0 (Moderate) upgrades applied:")
        for feature, count in sorted(a0_counts.items(), key=lambda x: x[1], reverse=True):
            self.logger.info(f"  {feature}: {count} homes ({count/len(a0_upgrades)*100:.1f}%)")
        
        self.logger.info("\nA1 (Ambitious) upgrades applied:")
        for feature, count in sorted(a1_counts.items(), key=lambda x: x[1], reverse=True):
            self.logger.info(f"  {feature}: {count} homes ({count/len(a1_upgrades)*100:.1f}%)")
        
        # Check for differences between A0 and A1
        all_features = set(a0_counts.keys()) | set(a1_counts.keys())
        differences = []
        for feature in all_features:
            a0_count = a0_counts.get(feature, 0)
            a1_count = a1_counts.get(feature, 0)
            if a1_count > a0_count:
                differences.append((feature, a1_count - a0_count))
        
        if differences:
            self.logger.info("\nAdditional upgrades in A1 vs A0:")
            for feature, diff in sorted(differences, key=lambda x: x[1], reverse=True):
                self.logger.info(f"  {feature}: +{diff} homes")
        else:
            self.logger.warning("\nWARNING: No difference between A0 and A1 upgrades!")
    
    def predict_scenario_outcomes(self, baseline_df: pd.DataFrame, 
                                 a0_df: pd.DataFrame, 
                                 a1_df: pd.DataFrame) -> None:
        """Use H2O model to predict EPC ratings for each scenario."""
        self.logger.info("Predicting outcomes for scenarios...")
        
        # Prepare features (exclude metadata columns)
        exclude_cols = [
            'CURRENT_ENERGY_RATING', 'cluster', 'matched_cluster',
            'distance_to_centroid', 'upgrades_needed', 'upgrade_priority_score',
            'upgrades_applied', 'scenario', 'ROOF_DESC_SCORE', 'WALLS_DESC_SCORE', 
            'FLOOR_DESC_SCORE'
        ]
        
        feature_cols = [col for col in baseline_df.columns if col not in exclude_cols]
        
        # Create H2O frames
        baseline_h2o = H2OFrame(baseline_df[feature_cols])
        a0_h2o = H2OFrame(a0_df[feature_cols])
        a1_h2o = H2OFrame(a1_df[feature_cols])
        
        # Ensure proper types
        categorical_cols = (self.config.categorical_features + 
                          self.available_numeric_fabric + 
                          self.available_service)
        
        for col in feature_cols:
            if col in categorical_cols:
                baseline_h2o[col] = baseline_h2o[col].asfactor()
                a0_h2o[col] = a0_h2o[col].asfactor()
                a1_h2o[col] = a1_h2o[col].asfactor()
        
        # Make predictions
        baseline_pred = self.h2o_model.predict(baseline_h2o)
        a0_pred = self.h2o_model.predict(a0_h2o)
        a1_pred = self.h2o_model.predict(a1_h2o)
        
        # Extract predictions - FIX: flatten the 2D array to 1D
        baseline_df['predicted_rating'] = baseline_pred['predict'].as_data_frame().values.ravel()
        a0_df['predicted_rating'] = a0_pred['predict'].as_data_frame().values.ravel()
        a1_df['predicted_rating'] = a1_pred['predict'].as_data_frame().values.ravel()
        
        # Get probability of achieving C or better
        baseline_pred_df = baseline_pred.as_data_frame()
        a0_pred_df = a0_pred.as_data_frame()
        a1_pred_df = a1_pred.as_data_frame()
        
        prob_cols = [col for col in baseline_pred_df.columns if col.startswith('p') or col in ['A','B','C','D','E','F','G']]
        
        if prob_cols:
            # Determine which columns represent A, B, C
            c_or_better_cols = []
            
            # Check for different possible naming conventions
            if 'A' in prob_cols and 'B' in prob_cols and 'C' in prob_cols:
                c_or_better_cols = ['A', 'B', 'C']
            elif 'pA' in prob_cols and 'pB' in prob_cols and 'pC' in prob_cols:
                c_or_better_cols = ['pA', 'pB', 'pC']
            else:
                # Try to identify based on column order
                sorted_prob_cols = sorted([col for col in prob_cols if col != 'predict'])
                if len(sorted_prob_cols) >= 7:  # Should have 7 classes (A-G)
                    c_or_better_cols = sorted_prob_cols[:3]  # First three are A, B, C
            
            if c_or_better_cols:
                # Calculate probability of C or better
                baseline_df['prob_c_or_better'] = baseline_pred_df[c_or_better_cols].sum(axis=1).values
                a0_df['prob_c_or_better'] = a0_pred_df[c_or_better_cols].sum(axis=1).values
                a1_df['prob_c_or_better'] = a1_pred_df[c_or_better_cols].sum(axis=1).values
            else:
                self.logger.warning("Could not identify probability columns for ratings A, B, C")
                baseline_df['prob_c_or_better'] = np.nan
                a0_df['prob_c_or_better'] = np.nan
                a1_df['prob_c_or_better'] = np.nan
        else:
            self.logger.warning("No probability columns found in predictions")
            baseline_df['prob_c_or_better'] = np.nan
            a0_df['prob_c_or_better'] = np.nan
            a1_df['prob_c_or_better'] = np.nan
        
        # Calculate improvements
        baseline_df['rating_improvement'] = 0
        a0_df['rating_improvement'] = self._calculate_rating_improvement(
            baseline_df['CURRENT_ENERGY_RATING'], 
            a0_df['predicted_rating']
        )
        a1_df['rating_improvement'] = self._calculate_rating_improvement(
            baseline_df['CURRENT_ENERGY_RATING'], 
            a1_df['predicted_rating']
        )
        
        # Store results
        self.baseline_df = baseline_df
        self.a0_scenario_df = a0_df
        self.a1_scenario_df = a1_df
        
        self.logger.info("Scenario predictions completed")
        self._log_prediction_summary()
    
    def _calculate_rating_improvement(self, current: pd.Series, predicted: pd.Series) -> pd.Series:
        """Calculate the number of rating bands improved."""
        rating_map = {'A': 7, 'B': 6, 'C': 5, 'D': 4, 'E': 3, 'F': 2, 'G': 1}
        
        current_numeric = current.map(rating_map)
        predicted_numeric = predicted.astype(str).map(rating_map)
        
        return predicted_numeric - current_numeric
    
    def _log_prediction_summary(self) -> None:
        """Log summary of prediction outcomes."""
        self.logger.info("\n" + "="*60)
        self.logger.info("PREDICTION SUMMARY")
        self.logger.info("="*60)
        
        # A0 Scenario outcomes
        self.logger.info("\nA0 (Moderate) Scenario:")
        a0_improved = (self.a0_scenario_df['rating_improvement'] > 0).sum()
        a0_achieved_c = (self.a0_scenario_df['predicted_rating'].isin(['A', 'B', 'C'])).sum()
        self.logger.info(f"  Homes improved: {a0_improved}/{len(self.a0_scenario_df)} "
                        f"({a0_improved/len(self.a0_scenario_df)*100:.1f}%)")
        self.logger.info(f"  Achieved C or better: {a0_achieved_c}/{len(self.a0_scenario_df)} "
                        f"({a0_achieved_c/len(self.a0_scenario_df)*100:.1f}%)")
        self.logger.info(f"  Mean improvement: {self.a0_scenario_df['rating_improvement'].mean():.2f} bands")
        
        # A1 Scenario outcomes
        self.logger.info("\nA1 (Ambitious) Scenario:")
        a1_improved = (self.a1_scenario_df['rating_improvement'] > 0).sum()
        a1_achieved_c = (self.a1_scenario_df['predicted_rating'].isin(['A', 'B', 'C'])).sum()
        self.logger.info(f"  Homes improved: {a1_improved}/{len(self.a1_scenario_df)} "
                        f"({a1_improved/len(self.a1_scenario_df)*100:.1f}%)")
        self.logger.info(f"  Achieved C or better: {a1_achieved_c}/{len(self.a1_scenario_df)} "
                        f"({a1_achieved_c/len(self.a1_scenario_df)*100:.1f}%)")
        self.logger.info(f"  Mean improvement: {self.a1_scenario_df['rating_improvement'].mean():.2f} bands")
        
        # Check if A0 and A1 are different
        if a0_improved == a1_improved and a0_achieved_c == a1_achieved_c:
            self.logger.warning("\nWARNING: A0 and A1 scenarios show identical outcomes!")
            self.logger.warning("This suggests upgrades may not be properly differentiating.")
    
    def calculate_costs_and_benefits(self) -> pd.DataFrame:
        """Calculate upgrade costs and energy savings for each scenario."""
        self.logger.info("Calculating costs and benefits...")
        
        results = []
        
        for idx in range(len(self.baseline_df)):
            baseline_row = self.baseline_df.iloc[idx]
            a0_row = self.a0_scenario_df.iloc[idx]
            a1_row = self.a1_scenario_df.iloc[idx]
            
            # Calculate costs with fixed glazing proportion
            a0_cost = self._calculate_upgrade_cost(
                baseline_row, a0_row, a0_row['upgrades_applied']
            )
            a1_cost = self._calculate_upgrade_cost(
                baseline_row, a1_row, a1_row['upgrades_applied']
            )
            
            # Estimate energy savings
            a0_savings = self._estimate_energy_savings(
                baseline_row['CURRENT_ENERGY_RATING'],
                a0_row['predicted_rating'],
                pd.to_numeric(baseline_row.get('TOTAL_FLOOR_AREA', 90), errors='coerce')
            )
            a1_savings = self._estimate_energy_savings(
                baseline_row['CURRENT_ENERGY_RATING'],
                a1_row['predicted_rating'],
                pd.to_numeric(baseline_row.get('TOTAL_FLOOR_AREA', 90), errors='coerce')
            )
            
            # Calculate payback periods
            a0_payback = a0_cost / a0_savings if a0_savings > 0 else np.inf
            a1_payback = a1_cost / a1_savings if a1_savings > 0 else np.inf
            
            results.append({
                'property_id': idx,
                'current_rating': baseline_row['CURRENT_ENERGY_RATING'],
                'cluster': baseline_row['matched_cluster'],
                
                'a0_predicted_rating': a0_row['predicted_rating'],
                'a0_improvement': a0_row['rating_improvement'],
                'a0_cost': a0_cost,
                'a0_annual_savings': a0_savings,
                'a0_payback_years': a0_payback,
                'a0_prob_c_or_better': a0_row.get('prob_c_or_better', np.nan),
                
                'a1_predicted_rating': a1_row['predicted_rating'],
                'a1_improvement': a1_row['rating_improvement'],
                'a1_cost': a1_cost,
                'a1_annual_savings': a1_savings,
                'a1_payback_years': a1_payback,
                'a1_prob_c_or_better': a1_row.get('prob_c_or_better', np.nan),
                
                'cost_per_improvement_a0': a0_cost / max(a0_row['rating_improvement'], 1),
                'cost_per_improvement_a1': a1_cost / max(a1_row['rating_improvement'], 1),
            })
        
        self.cost_benefit_df = pd.DataFrame(results)
        self._log_cost_benefit_summary()
        
        return self.cost_benefit_df
    
    def _calculate_upgrade_cost(self, baseline: pd.Series, upgraded: pd.Series, 
                               upgrades: Dict) -> float:
        """Calculate the cost of upgrades for a single home with FIXED scaling issues."""
        total_cost = 0.0
        
        # Get floor area and validate
        floor_area = pd.to_numeric(baseline.get('TOTAL_FLOOR_AREA', 90), errors='coerce')
        if pd.isna(floor_area) or floor_area <= 0:
            floor_area = 90  # UK average
        
        # Cap floor area to reasonable range (20-500 sqm)
        floor_area = np.clip(floor_area, 20, 500)
        
        # Estimate areas
        wall_area = floor_area * 2.5  # Typical wall/floor ratio
        
        # CRITICAL FIX: Handle MULTI_GLAZE_PROPORTION correctly
        glazing_ratio = pd.to_numeric(baseline.get('MULTI_GLAZE_PROPORTION', 15), errors='coerce')
        
        # If it's stored as percentage (0-100), convert to ratio (0-1)
        if pd.notna(glazing_ratio) and glazing_ratio > 1:
            glazing_ratio = glazing_ratio / 100.0
        
        # Ensure reasonable bounds (5-40% of wall area)
        glazing_ratio = float(np.clip(glazing_ratio if pd.notna(glazing_ratio) else 0.15, 0.05, 0.40))
        glazing_area = wall_area * glazing_ratio
        
        if isinstance(upgrades, dict):
            # Wall insulation - only charge if efficiency actually changed
            if upgrades.get('WALLS_ENERGY_EFF', False):
                wall_desc = str(baseline.get('WALLS_DESCRIPTION', '')).lower()
                if 'cavity' in wall_desc:
                    total_cost += wall_area * self.config.costs['wall_insulation_cavity']
                else:
                    total_cost += wall_area * self.config.costs['wall_insulation_solid']
            
            # Roof insulation
            if upgrades.get('ROOF_ENERGY_EFF', False):
                roof_area = floor_area * 0.9  # Approximate roof area
                total_cost += roof_area * self.config.costs['roof_insulation']
            
            # Floor insulation
            if upgrades.get('FLOOR_ENERGY_EFF', False):
                total_cost += floor_area * self.config.costs['floor_insulation']
            
            # Windows
            if upgrades.get('WINDOWS_ENERGY_EFF', False):
                total_cost += glazing_area * self.config.costs['double_glazing']
            
            # Heating system
            if upgrades.get('MAINHEAT_ENERGY_EFF', False):
                total_cost += self.config.costs['heating_system_upgrade']
            
            # Hot water system
            if upgrades.get('HOT_WATER_ENERGY_EFF', False):
                total_cost += self.config.costs['hot_water_upgrade']
            
            # Lighting
            if upgrades.get('LIGHTING_ENERGY_EFF', False) or upgrades.get('LOW_ENERGY_LIGHTING', False):
                total_cost += self.config.costs['lighting_upgrade']
        
        return total_cost
    
    def _estimate_energy_savings(self, current_rating: str, predicted_rating: str, 
                                floor_area: float = 90) -> float:
        """Estimate annual energy cost savings from rating improvement with size scaling."""
        # Base consumption by rating (for 90 sqm reference home)
        consumption_by_rating = {
            'A': 8000,   # kWh/year
            'B': 12000,
            'C': 16000,
            'D': 20000,
            'E': 24000,
            'F': 28000,
            'G': 32000
        }
        
        # Scale by floor area
        if pd.isna(floor_area) or floor_area <= 0:
            floor_area = 90
        size_factor = np.clip((floor_area / 90.0), 0.5, 2.0)  # 90 m² reference
        
        current_consumption = consumption_by_rating.get(current_rating, 25000) * size_factor
        predicted_consumption = consumption_by_rating.get(predicted_rating, current_consumption) * size_factor
        
        kwh_saved = current_consumption - predicted_consumption
        annual_savings = kwh_saved * self.config.energy_price_per_kwh
        
        return max(0, annual_savings)
    
    def _log_cost_benefit_summary(self) -> None:
        """Log summary of cost-benefit analysis."""
        self.logger.info("\n" + "="*60)
        self.logger.info("COST-BENEFIT SUMMARY")
        self.logger.info("="*60)
        
        # A0 Scenario
        self.logger.info("\nA0 (Moderate) Scenario:")
        self.logger.info(f"  Average cost: £{self.cost_benefit_df['a0_cost'].mean():,.2f}")
        self.logger.info(f"  Median cost: £{self.cost_benefit_df['a0_cost'].median():,.2f}")
        self.logger.info(f"  Average annual savings: £{self.cost_benefit_df['a0_annual_savings'].mean():,.2f}")
        
        payback_finite = self.cost_benefit_df[self.cost_benefit_df['a0_payback_years'] < 50]['a0_payback_years']
        if len(payback_finite) > 0:
            self.logger.info(f"  Average payback period: {payback_finite.mean():.1f} years")
            self.logger.info(f"  Median payback period: {payback_finite.median():.1f} years")
        
        self.logger.info(f"  Cost per rating improvement: £{self.cost_benefit_df['cost_per_improvement_a0'].mean():,.2f}")
        
        # A1 Scenario
        self.logger.info("\nA1 (Ambitious) Scenario:")
        self.logger.info(f"  Average cost: £{self.cost_benefit_df['a1_cost'].mean():,.2f}")
        self.logger.info(f"  Median cost: £{self.cost_benefit_df['a1_cost'].median():,.2f}")
        self.logger.info(f"  Average annual savings: £{self.cost_benefit_df['a1_annual_savings'].mean():,.2f}")
        
        payback_finite = self.cost_benefit_df[self.cost_benefit_df['a1_payback_years'] < 50]['a1_payback_years']
        if len(payback_finite) > 0:
            self.logger.info(f"  Average payback period: {payback_finite.mean():.1f} years")
            self.logger.info(f"  Median payback period: {payback_finite.median():.1f} years")
        
        self.logger.info(f"  Cost per rating improvement: £{self.cost_benefit_df['cost_per_improvement_a1'].mean():,.2f}")
        
        # Check for reasonable values
        if self.cost_benefit_df['a0_cost'].mean() > 100000:
            self.logger.warning("\nWARNING: Average costs still seem too high! Check floor area and glazing data.")
        
        if abs(self.cost_benefit_df['a0_cost'].mean() - self.cost_benefit_df['a1_cost'].mean()) < 100:
            self.logger.warning("\nWARNING: A0 and A1 costs are nearly identical! Check upgrade differentiation.")
    
    def calculate_cost_by_rating_band(self) -> pd.DataFrame:
        """Calculate average upgrade costs by initial rating band."""
        self.logger.info("Calculating average costs by rating band...")
        
        # Initialize results dictionary
        rating_analysis = {
            'rating': [],
            'total_homes': [],
            'a0_avg_cost': [],
            'a0_success_rate': [],
            'a0_avg_cost_per_success': [],
            'a1_avg_cost': [],
            'a1_success_rate': [],
            'a1_avg_cost_per_success': [],
        }
        
        # Analyze each sub-standard rating
        for rating in ['D', 'E', 'F', 'G']:
            # Filter homes with this rating
            mask = self.cost_benefit_df['current_rating'] == rating
            rating_homes = self.cost_benefit_df[mask]
            
            if len(rating_homes) == 0:
                continue
                
            # Calculate A0 metrics
            a0_success = rating_homes['a0_predicted_rating'].isin(['A', 'B', 'C'])
            a0_success_rate = a0_success.mean()
            a0_avg_cost = rating_homes['a0_cost'].mean()
            
            # Cost per successful upgrade (only for homes that reached C or better)
            if a0_success.sum() > 0:
                a0_cost_per_success = rating_homes[a0_success]['a0_cost'].mean()
            else:
                a0_cost_per_success = np.nan
                
            # Calculate A1 metrics  
            a1_success = rating_homes['a1_predicted_rating'].isin(['A', 'B', 'C'])
            a1_success_rate = a1_success.mean()
            a1_avg_cost = rating_homes['a1_cost'].mean()
            
            # Cost per successful upgrade
            if a1_success.sum() > 0:
                a1_cost_per_success = rating_homes[a1_success]['a1_cost'].mean()
            else:
                a1_cost_per_success = np.nan
                
            # Store results
            rating_analysis['rating'].append(rating)
            rating_analysis['total_homes'].append(len(rating_homes))
            rating_analysis['a0_avg_cost'].append(a0_avg_cost)
            rating_analysis['a0_success_rate'].append(a0_success_rate)
            rating_analysis['a0_avg_cost_per_success'].append(a0_cost_per_success)
            rating_analysis['a1_avg_cost'].append(a1_avg_cost)
            rating_analysis['a1_success_rate'].append(a1_success_rate)
            rating_analysis['a1_avg_cost_per_success'].append(a1_cost_per_success)
        
        # Create DataFrame
        rating_cost_df = pd.DataFrame(rating_analysis)
        
        # Add difficulty metrics (how much harder/costlier for worse ratings)
        if len(rating_cost_df) > 0:
            # Use D as baseline
            d_idx = rating_cost_df[rating_cost_df['rating'] == 'D'].index
            if len(d_idx) > 0:
                d_idx = d_idx[0]
                d_a0_cost = rating_cost_df.loc[d_idx, 'a0_avg_cost']
                d_a1_cost = rating_cost_df.loc[d_idx, 'a1_avg_cost']
                
                rating_cost_df['a0_cost_ratio_to_D'] = rating_cost_df['a0_avg_cost'] / d_a0_cost
                rating_cost_df['a1_cost_ratio_to_D'] = rating_cost_df['a1_avg_cost'] / d_a1_cost
        
        # Log the analysis
        self._log_rating_band_analysis(rating_cost_df)
        
        # Save to results
        rating_cost_df.to_csv(
            self.results_dir / f'cost_by_rating_band_{self.run_timestamp}.csv',
            index=False
        )
        
        self.rating_cost_df = rating_cost_df
        return rating_cost_df
    
    def _log_rating_band_analysis(self, df: pd.DataFrame) -> None:
        """Log the rating band cost analysis."""
        self.logger.info("\n" + "="*60)
        self.logger.info("COST ANALYSIS BY INITIAL RATING")
        self.logger.info("="*60)
        
        for _, row in df.iterrows():
            self.logger.info(f"\nRating {row['rating']} → C:")
            self.logger.info(f"  Total homes: {int(row['total_homes'])}")
            
            self.logger.info(f"  A0 (Moderate):")
            self.logger.info(f"    Average cost: £{row['a0_avg_cost']:,.2f}")
            self.logger.info(f"    Success rate: {row['a0_success_rate']*100:.1f}%")
            if not pd.isna(row['a0_avg_cost_per_success']):
                self.logger.info(f"    Cost per success: £{row['a0_avg_cost_per_success']:,.2f}")
            
            self.logger.info(f"  A1 (Ambitious):")
            self.logger.info(f"    Average cost: £{row['a1_avg_cost']:,.2f}")
            self.logger.info(f"    Success rate: {row['a1_success_rate']*100:.1f}%")
            if not pd.isna(row['a1_avg_cost_per_success']):
                self.logger.info(f"    Cost per success: £{row['a1_avg_cost_per_success']:,.2f}")
        
        # Summary insights
        self.logger.info("\nKEY INSIGHTS:")
        
        # Cost progression
        if 'a0_cost_ratio_to_D' in df.columns:
            self.logger.info("\nCost multipliers relative to D→C upgrade:")
            for _, row in df.iterrows():
                if row['rating'] != 'D' and not pd.isna(row.get('a0_cost_ratio_to_D')):
                    self.logger.info(f"  {row['rating']}→C costs {row['a0_cost_ratio_to_D']:.1f}x "
                                   f"more than D→C (A0)")
        
        # Most cost-effective starting point
        if len(df) > 0:
            best_value_a0_idx = df['a0_avg_cost_per_success'].idxmin()
            best_value_a1_idx = df['a1_avg_cost_per_success'].idxmin()
            
            if pd.notna(best_value_a0_idx):
                best_value_a0 = df.loc[best_value_a0_idx]
                self.logger.info(f"\nMost cost-effective upgrades:")
                self.logger.info(f"  A0: {best_value_a0['rating']}→C "
                                f"(£{best_value_a0['a0_avg_cost_per_success']:,.0f} per success)")
            
            if pd.notna(best_value_a1_idx):
                best_value_a1 = df.loc[best_value_a1_idx]
                self.logger.info(f"  A1: {best_value_a1['rating']}→C "
                                f"(£{best_value_a1['a1_avg_cost_per_success']:,.0f} per success)")
    
    def visualize_cost_by_rating(self, rating_cost_df: pd.DataFrame) -> None:
        """Create visualization of costs by initial rating band."""
        import matplotlib.pyplot as plt
        import seaborn as sns
        
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        # Sort by rating order
        rating_order = ['D', 'E', 'F', 'G']
        rating_cost_df = rating_cost_df.set_index('rating').reindex(rating_order).reset_index()
        
        # 1. Average costs comparison
        ax = axes[0, 0]
        x = np.arange(len(rating_cost_df))
        width = 0.35
        
        ax.bar(x - width/2, rating_cost_df['a0_avg_cost'], width, 
               label='A0 (Moderate)', color='steelblue', alpha=0.8)
        ax.bar(x + width/2, rating_cost_df['a1_avg_cost'], width,
               label='A1 (Ambitious)', color='coral', alpha=0.8)
        
        ax.set_xlabel('Initial Rating')
        ax.set_ylabel('Average Upgrade Cost (£)')
        ax.set_title('Average Cost to Achieve C by Initial Rating')
        ax.set_xticks(x)
        ax.set_xticklabels(rating_cost_df['rating'])
        ax.legend()
        ax.grid(True, alpha=0.3, axis='y')
        
        # Add value labels on bars
        for i, (a0, a1) in enumerate(zip(rating_cost_df['a0_avg_cost'], 
                                         rating_cost_df['a1_avg_cost'])):
            ax.text(i - width/2, a0 + 500, f'£{a0:,.0f}', ha='center', va='bottom', fontsize=9)
            ax.text(i + width/2, a1 + 500, f'£{a1:,.0f}', ha='center', va='bottom', fontsize=9)
        
        # 2. Success rates
        ax = axes[0, 1]
        ax.bar(x - width/2, rating_cost_df['a0_success_rate'] * 100, width,
               label='A0 (Moderate)', color='steelblue', alpha=0.8)
        ax.bar(x + width/2, rating_cost_df['a1_success_rate'] * 100, width,
               label='A1 (Ambitious)', color='coral', alpha=0.8)
        
        ax.set_xlabel('Initial Rating')
        ax.set_ylabel('Success Rate (%)')
        ax.set_title('Success Rate in Achieving C or Better')
        ax.set_xticks(x)
        ax.set_xticklabels(rating_cost_df['rating'])
        ax.legend()
        ax.grid(True, alpha=0.3, axis='y')
        ax.set_ylim(0, 105)
        
        # 3. Cost per successful upgrade
        ax = axes[1, 0]
        ax.bar(x - width/2, rating_cost_df['a0_avg_cost_per_success'], width,
               label='A0 (Moderate)', color='steelblue', alpha=0.8)
        ax.bar(x + width/2, rating_cost_df['a1_avg_cost_per_success'], width,
               label='A1 (Ambitious)', color='coral', alpha=0.8)
        
        ax.set_xlabel('Initial Rating')
        ax.set_ylabel('Cost per Successful Upgrade (£)')
        ax.set_title('Cost Efficiency: £ per Home Successfully Upgraded to C')
        ax.set_xticks(x)
        ax.set_xticklabels(rating_cost_df['rating'])
        ax.legend()
        ax.grid(True, alpha=0.3, axis='y')
        
        # 4. Number of homes per rating
        ax = axes[1, 1]
        colors = ['#2E7D32', '#558B2F', '#827717', '#BF360C']  # Green to red gradient
        bars = ax.bar(rating_cost_df['rating'], rating_cost_df['total_homes'], 
                      color=colors, alpha=0.8)
        
        ax.set_xlabel('Initial Rating')
        ax.set_ylabel('Number of Homes')
        ax.set_title('Distribution of Homes by Initial Rating')
        ax.grid(True, alpha=0.3, axis='y')
        
        # Add percentage labels
        total = rating_cost_df['total_homes'].sum()
        for bar, count in zip(bars, rating_cost_df['total_homes']):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 5,
                    f'{count}\n({count/total*100:.1f}%)',
                    ha='center', va='bottom')
        
        plt.suptitle('Upgrade Cost Analysis by Initial EPC Rating', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.plots_dir / 'cost_analysis_by_rating.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        self.logger.info("Rating band cost visualization saved")
    
    def generate_visualizations(self) -> None:
        """Generate comprehensive visualizations of results."""
        import matplotlib.pyplot as plt
        import seaborn as sns
        
        plt.style.use('default')
        sns.set_palette("husl")
        
        # 1. Rating improvement distribution
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        # A0 improvements
        improvement_counts = self.a0_scenario_df['rating_improvement'].value_counts().sort_index()
        ax1.bar(improvement_counts.index, improvement_counts.values, alpha=0.8, color='steelblue')
        ax1.set_xlabel('Rating Bands Improved')
        ax1.set_ylabel('Number of Homes')
        ax1.set_title('A0 (Moderate) - Rating Improvements')
        ax1.grid(True, alpha=0.3)
        
        # A1 improvements
        improvement_counts = self.a1_scenario_df['rating_improvement'].value_counts().sort_index()
        ax2.bar(improvement_counts.index, improvement_counts.values, alpha=0.8, color='coral')
        ax2.set_xlabel('Rating Bands Improved')
        ax2.set_ylabel('Number of Homes')
        ax2.set_title('A1 (Ambitious) - Rating Improvements')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(self.plots_dir / 'rating_improvements.png', dpi=300)
        plt.close()
        
        # 2. Cost vs Benefit scatter (with reasonable cost bounds)
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        # Filter for reasonable costs (under £100k) and payback (under 50 years)
        mask_a0 = (self.cost_benefit_df['a0_cost'] < 100000) & (self.cost_benefit_df['a0_payback_years'] < 50)
        mask_a1 = (self.cost_benefit_df['a1_cost'] < 100000) & (self.cost_benefit_df['a1_payback_years'] < 50)
        
        # A0 cost-benefit
        if mask_a0.sum() > 0:
            ax1.scatter(self.cost_benefit_df[mask_a0]['a0_cost'], 
                       self.cost_benefit_df[mask_a0]['a0_annual_savings'],
                       alpha=0.6, s=20, color='steelblue')
            ax1.set_xlabel('Upgrade Cost (£)')
            ax1.set_ylabel('Annual Savings (£)')
            ax1.set_title('A0 - Cost vs Annual Savings')
            ax1.grid(True, alpha=0.3)
            
            # Add payback lines
            x_max = self.cost_benefit_df[mask_a0]['a0_cost'].max()
            for years in [5, 10, 20]:
                x = np.linspace(0, x_max, 100)
                y = x / years
                ax1.plot(x, y, '--', alpha=0.5, label=f'{years}yr payback')
            ax1.legend()
        
        # A1 cost-benefit
        if mask_a1.sum() > 0:
            ax2.scatter(self.cost_benefit_df[mask_a1]['a1_cost'], 
                       self.cost_benefit_df[mask_a1]['a1_annual_savings'],
                       alpha=0.6, s=20, color='coral')
            ax2.set_xlabel('Upgrade Cost (£)')
            ax2.set_ylabel('Annual Savings (£)')
            ax2.set_title('A1 - Cost vs Annual Savings')
            ax2.grid(True, alpha=0.3)
            
            # Add payback lines
            x_max = self.cost_benefit_df[mask_a1]['a1_cost'].max()
            for years in [5, 10, 20]:
                x = np.linspace(0, x_max, 100)
                y = x / years
                ax2.plot(x, y, '--', alpha=0.5, label=f'{years}yr payback')
            ax2.legend()
        
        plt.tight_layout()
        plt.savefig(self.plots_dir / 'cost_benefit_analysis.png', dpi=300)
        plt.close()
        
        # 3. Achievement rates by cluster
        fig, ax = plt.subplots(figsize=(10, 6))
        
        clusters = sorted(self.cost_benefit_df['cluster'].unique())
        a0_rates = []
        a1_rates = []
        
        for cluster in clusters:
            mask = self.cost_benefit_df['cluster'] == cluster
            a0_rate = (self.cost_benefit_df[mask]['a0_predicted_rating'].isin(['A', 'B', 'C'])).mean() * 100
            a1_rate = (self.cost_benefit_df[mask]['a1_predicted_rating'].isin(['A', 'B', 'C'])).mean() * 100
            a0_rates.append(a0_rate)
            a1_rates.append(a1_rate)
        
        x = np.arange(len(clusters))
        width = 0.35
        
        ax.bar(x - width/2, a0_rates, width, label='A0 (Moderate)', alpha=0.8, color='steelblue')
        ax.bar(x + width/2, a1_rates, width, label='A1 (Ambitious)', alpha=0.8, color='coral')
        
        ax.set_xlabel('Cluster')
        ax.set_ylabel('% Achieving C or Better')
        ax.set_title('Success Rate by Cluster and Scenario')
        ax.set_xticks(x)
        ax.set_xticklabels([f'Cluster {c}' for c in clusters])
        ax.legend()
        ax.grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.savefig(self.plots_dir / 'success_by_cluster.png', dpi=300)
        plt.close()
        
        self.logger.info("Visualizations generated successfully")
    
    def save_results(self) -> None:
        """Save all analysis results."""
        self.logger.info("Saving results...")
        
        # Save scenario data
        self.baseline_df.to_csv(
            self.results_dir / f'baseline_scenario_{self.run_timestamp}.csv',
            index=False
        )
        self.a0_scenario_df.to_csv(
            self.results_dir / f'a0_moderate_scenario_{self.run_timestamp}.csv',
            index=False
        )
        self.a1_scenario_df.to_csv(
            self.results_dir / f'a1_ambitious_scenario_{self.run_timestamp}.csv',
            index=False
        )
        
        # Save cost-benefit analysis
        self.cost_benefit_df.to_csv(
            self.results_dir / f'cost_benefit_analysis_{self.run_timestamp}.csv',
            index=False
        )
        
        # Save summary statistics
        summary_stats = self._generate_summary_statistics()
        with open(self.results_dir / f'summary_statistics_{self.run_timestamp}.json', 'w') as f:
            json.dump(summary_stats, f, indent=2)
        
        self.logger.info(f"All results saved to: {self.run_output_dir}")
    
    def _generate_summary_statistics(self) -> Dict[str, Any]:
        """Generate comprehensive summary statistics."""
        # Calculate reasonable payback statistics (exclude infinite values)
        a0_payback_finite = self.cost_benefit_df[self.cost_benefit_df['a0_payback_years'] < 50]['a0_payback_years']
        a1_payback_finite = self.cost_benefit_df[self.cost_benefit_df['a1_payback_years'] < 50]['a1_payback_years']
        
        summary = {
            'timestamp': self.run_timestamp,
            'total_homes_analyzed': len(self.baseline_df),
            'available_features': {
                'numeric_fabric': self.available_numeric_fabric,
                'service': self.available_service,
                'categorical_fabric': self.available_categorical_fabric
            },
            'feature_importance_loaded': self.feature_importance is not None,
            'scenarios': {
                'A0_moderate': {
                    'homes_improved': int((self.a0_scenario_df['rating_improvement'] > 0).sum()),
                    'achieved_c_or_better': int((self.a0_scenario_df['predicted_rating'].isin(['A', 'B', 'C'])).sum()),
                    'average_improvement': float(self.a0_scenario_df['rating_improvement'].mean()),
                    'average_cost': float(self.cost_benefit_df['a0_cost'].mean()),
                    'median_cost': float(self.cost_benefit_df['a0_cost'].median()),
                    'average_annual_savings': float(self.cost_benefit_df['a0_annual_savings'].mean()),
                    'average_payback_years': float(a0_payback_finite.mean()) if len(a0_payback_finite) > 0 else None,
                    'median_payback_years': float(a0_payback_finite.median()) if len(a0_payback_finite) > 0 else None,
                },
                'A1_ambitious': {
                    'homes_improved': int((self.a1_scenario_df['rating_improvement'] > 0).sum()),
                    'achieved_c_or_better': int((self.a1_scenario_df['predicted_rating'].isin(['A', 'B', 'C'])).sum()),
                    'average_improvement': float(self.a1_scenario_df['rating_improvement'].mean()),
                    'average_cost': float(self.cost_benefit_df['a1_cost'].mean()),
                    'median_cost': float(self.cost_benefit_df['a1_cost'].median()),
                    'average_annual_savings': float(self.cost_benefit_df['a1_annual_savings'].mean()),
                    'average_payback_years': float(a1_payback_finite.mean()) if len(a1_payback_finite) > 0 else None,
                    'median_payback_years': float(a1_payback_finite.median()) if len(a1_payback_finite) > 0 else None,
                }
            },
            'model_info': {
                'model_id': self.h2o_model.model_id if self.h2o_model else None,
                'model_type': type(self.h2o_model).__name__ if self.h2o_model else None,
            }
        }
        
        # Add rating band analysis if available
        if hasattr(self, 'rating_cost_df') and self.rating_cost_df is not None:
            summary['rating_band_analysis'] = self.rating_cost_df.to_dict('records')
        
        return summary
    
    def generate_report(self) -> None:
        """Generate comprehensive final report."""
        report_path = self.run_output_dir / f'scenario_analysis_report_{self.run_timestamp}.txt'
        
        with open(report_path, 'w') as f:
            f.write("="*80 + "\n")
            f.write("EPC UPGRADE SCENARIO ANALYSIS REPORT\n")
            f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("="*80 + "\n\n")
            
            # Executive Summary
            f.write("EXECUTIVE SUMMARY\n")
            f.write("-"*40 + "\n")
            f.write(f"Total homes analyzed: {len(self.baseline_df)}\n")
            f.write(f"Current ratings distribution:\n")
            for rating in ['D', 'E', 'F', 'G']:
                count = (self.baseline_df['CURRENT_ENERGY_RATING'] == rating).sum()
                f.write(f"  {rating}: {count} ({count/len(self.baseline_df)*100:.1f}%)\n")
            f.write("\n")
            
            # Feature importance status
            f.write("FEATURE IMPORTANCE\n")
            f.write("-"*40 + "\n")
            if self.feature_importance is not None:
                f.write(f"Successfully loaded feature importance from Step 1\n")
                f.write(f"Top 5 most important features:\n")
                for i, row in self.feature_importance.head(5).iterrows():
                    f.write(f"  - {row['feature']}: {row.get('mean_importance', 0):.2f}\n")
            else:
                f.write("Feature importance not available (treating all features equally)\n")
            f.write("\n")
            
            # Scenario outcomes
            f.write("SCENARIO OUTCOMES\n")
            f.write("-"*40 + "\n")
            
            f.write("A0 (Moderate) Scenario:\n")
            a0_success = (self.a0_scenario_df['predicted_rating'].isin(['A', 'B', 'C'])).sum()
            f.write(f"  Homes achieving C or better: {a0_success}/{len(self.a0_scenario_df)} "
                   f"({a0_success/len(self.a0_scenario_df)*100:.1f}%)\n")
            f.write(f"  Average cost: £{self.cost_benefit_df['a0_cost'].mean():,.0f}\n")
            f.write(f"  Median cost: £{self.cost_benefit_df['a0_cost'].median():,.0f}\n")
            f.write(f"  Average annual savings: £{self.cost_benefit_df['a0_annual_savings'].mean():,.0f}\n")
            payback_finite = self.cost_benefit_df[self.cost_benefit_df['a0_payback_years'] < 50]['a0_payback_years']
            if len(payback_finite) > 0:
                f.write(f"  Average payback: {payback_finite.mean():.1f} years\n")
                f.write(f"  Median payback: {payback_finite.median():.1f} years\n")
            f.write("\n")
            
            f.write("A1 (Ambitious) Scenario:\n")
            a1_success = (self.a1_scenario_df['predicted_rating'].isin(['A', 'B', 'C'])).sum()
            f.write(f"  Homes achieving C or better: {a1_success}/{len(self.a1_scenario_df)} "
                   f"({a1_success/len(self.a1_scenario_df)*100:.1f}%)\n")
            f.write(f"  Average cost: £{self.cost_benefit_df['a1_cost'].mean():,.0f}\n")
            f.write(f"  Median cost: £{self.cost_benefit_df['a1_cost'].median():,.0f}\n")
            f.write(f"  Average annual savings: £{self.cost_benefit_df['a1_annual_savings'].mean():,.0f}\n")
            payback_finite = self.cost_benefit_df[self.cost_benefit_df['a1_payback_years'] < 50]['a1_payback_years']
            if len(payback_finite) > 0:
                f.write(f"  Average payback: {payback_finite.mean():.1f} years\n")
                f.write(f"  Median payback: {payback_finite.median():.1f} years\n")
            f.write("\n")
            
            # Rating band analysis (NEW)
            if hasattr(self, 'rating_cost_df') and self.rating_cost_df is not None:
                f.write("COST ANALYSIS BY INITIAL RATING\n")
                f.write("-"*40 + "\n")
                for _, row in self.rating_cost_df.iterrows():
                    f.write(f"\n{row['rating']} → C upgrade:\n")
                    f.write(f"  Number of homes: {int(row['total_homes'])}\n")
                    f.write(f"  A0 average cost: £{row['a0_avg_cost']:,.0f}")
                    f.write(f" (success rate: {row['a0_success_rate']*100:.1f}%)\n")
                    f.write(f"  A1 average cost: £{row['a1_avg_cost']:,.0f}")
                    f.write(f" (success rate: {row['a1_success_rate']*100:.1f}%)\n")
                f.write("\n")
            
            # Key findings
            f.write("KEY FINDINGS\n")
            f.write("-"*40 + "\n")
            
            # Cost effectiveness
            cost_effective_a0 = (self.cost_benefit_df['a0_payback_years'] <= 10).sum()
            cost_effective_a1 = (self.cost_benefit_df['a1_payback_years'] <= 10).sum()
            
            f.write(f"1. Cost-effectiveness (≤10 year payback):\n")
            f.write(f"   - A0: {cost_effective_a0} homes ({cost_effective_a0/len(self.cost_benefit_df)*100:.1f}%)\n")
            f.write(f"   - A1: {cost_effective_a1} homes ({cost_effective_a1/len(self.cost_benefit_df)*100:.1f}%)\n\n")
            
            # Improvement differential
            a0_improved = (self.a0_scenario_df['rating_improvement'] > 0).sum()
            a1_improved = (self.a1_scenario_df['rating_improvement'] > 0).sum()
            f.write(f"2. Improvement differential:\n")
            f.write(f"   - A0 improved: {a0_improved} homes\n")
            f.write(f"   - A1 improved: {a1_improved} homes\n")
            f.write(f"   - Additional in A1: {a1_improved - a0_improved} homes\n\n")
            
            # Data quality notes
            f.write("DATA QUALITY NOTES\n")
            f.write("-"*40 + "\n")
            floor_areas = pd.to_numeric(self.baseline_df['TOTAL_FLOOR_AREA'], errors='coerce')
            f.write(f"Floor area mean: {floor_areas.mean():.2f} sqm\n")
            if floor_areas.mean() > 500:
                f.write("WARNING: Floor areas may need unit conversion\n")
            if 'MULTI_GLAZE_PROPORTION' in self.baseline_df.columns:
                glazing = pd.to_numeric(self.baseline_df['MULTI_GLAZE_PROPORTION'], errors='coerce')
                if glazing.mean() > 1:
                    f.write(f"Glazing proportion mean: {glazing.mean():.2f} (converted from %)\n")
            f.write("\n")
            
            # Files generated
            f.write("FILES GENERATED\n")
            f.write("-"*40 + "\n")
            f.write(f"Output directory: {self.run_output_dir}\n")
            f.write("- Scenario data files (baseline, A0, A1)\n")
            f.write("- Cost-benefit analysis\n")
            f.write("- Cost by rating band analysis\n")
            f.write("- Visualization plots\n")
            f.write("- Summary statistics (JSON)\n")
            f.write("\n")
            
            f.write("="*80 + "\n")
            f.write("END OF REPORT\n")
            f.write("="*80 + "\n")
        
        self.logger.info(f"Report saved to: {report_path}")
    
    def run_analysis(self) -> Dict[str, Any]:
        """Run the complete scenario generation and analysis pipeline."""
        self.logger.info("Starting Scenario Generation and Impact Prediction...")
        self.logger.info("="*60)
        
        try:
            # 1. Initialize H2O and load model
            self.logger.info("Step 1: Initializing H2O and loading model...")
            self.init_h2o()
            
            # 2. Load Step 2 clustering data
            self.logger.info("Step 2: Loading clustering results...")
            self.epc_c_data, self.substandard_data = self.load_step2_data()
            
            # 2.5. Run diagnostics
            self.logger.info("Step 2.5: Running data diagnostics...")
            self.diagnose_data()
            
            # 3. Compute upgrade targets
            self.logger.info("Step 3: Computing upgrade targets...")
            targets = self.compute_upgrade_targets(self.epc_c_data)
            
            # 4. Create upgrade scenarios
            self.logger.info("Step 4: Creating upgrade scenarios...")
            baseline_df, a0_df, a1_df = self.create_upgrade_scenarios(
                self.substandard_data, targets
            )
            
            # 5. Predict outcomes
            self.logger.info("Step 5: Predicting scenario outcomes...")
            self.predict_scenario_outcomes(baseline_df, a0_df, a1_df)
            
            # 6. Calculate costs and benefits
            self.logger.info("Step 6: Calculating costs and benefits...")
            self.calculate_costs_and_benefits()
            
            # 6.5 Calculate costs by rating band (NEW)
            self.logger.info("Step 6.5: Analyzing costs by rating band...")
            rating_cost_df = self.calculate_cost_by_rating_band()
            self.visualize_cost_by_rating(rating_cost_df)
            
            # 7. Generate visualizations
            self.logger.info("Step 7: Generating visualizations...")
            self.generate_visualizations()
            
            # 8. Save results
            self.logger.info("Step 8: Saving results...")
            self.save_results()
            
            # 9. Generate report
            self.logger.info("Step 9: Generating final report...")
            self.generate_report()
            
            self.logger.info("\n" + "="*60)
            self.logger.info("SCENARIO ANALYSIS COMPLETE")
            self.logger.info("="*60)
            self.logger.info(f"Results saved to: {self.run_output_dir}")
            
            # Shutdown H2O
            h2o.shutdown(prompt=False)
            
            return {
                "status": "success",
                "homes_analyzed": len(self.baseline_df),
                "a0_success_rate": (self.a0_scenario_df['predicted_rating'].isin(['A', 'B', 'C'])).mean(),
                "a1_success_rate": (self.a1_scenario_df['predicted_rating'].isin(['A', 'B', 'C'])).mean(),
                "a0_avg_cost": self.cost_benefit_df['a0_cost'].mean(),
                "a1_avg_cost": self.cost_benefit_df['a1_cost'].mean(),
                "feature_importance_loaded": self.feature_importance is not None,
                "rating_band_analysis_completed": self.rating_cost_df is not None,
                "output_directory": str(self.run_output_dir)
            }
            
        except Exception as e:
            self.logger.error(f"Analysis failed: {e}")
            import traceback
            traceback.print_exc()
            
            # Try to shutdown H2O
            try:
                h2o.shutdown(prompt=False)
            except:
                pass
            
            return {
                "status": "failed",
                "error": str(e),
                "output_directory": str(self.run_output_dir) if hasattr(self, 'run_output_dir') else None
            }


def main():
    """Main execution function."""
    config = Config()
    generator = ScenarioGenerator(config)
    
    try:
        results = generator.run_analysis()
        
        if results["status"] == "success":
            print(f"\n{'='*60}")
            print("SCENARIO ANALYSIS COMPLETED SUCCESSFULLY!")
            print(f"{'='*60}")
            print(f"Homes analyzed: {results['homes_analyzed']}")
            print(f"A0 success rate: {results['a0_success_rate']*100:.1f}%")
            print(f"A1 success rate: {results['a1_success_rate']*100:.1f}%")
            print(f"A0 average cost: £{results['a0_avg_cost']:,.2f}")
            print(f"A1 average cost: £{results['a1_avg_cost']:,.2f}")
            print(f"Feature importance loaded: {results['feature_importance_loaded']}")
            print(f"Rating band analysis completed: {results['rating_band_analysis_completed']}")
            print(f"Results saved to: {results['output_directory']}")
            print(f"{'='*60}")
        else:
            print(f"\nAnalysis failed: {results['error']}")
            
    except KeyboardInterrupt:
        print("\nAnalysis interrupted by user")
        try:
            h2o.shutdown(prompt=False)
        except:
            pass
    except Exception as e:
        print(f"\nUnexpected error: {e}")
        import traceback
        traceback.print_exc()
        try:
            h2o.shutdown(prompt=False)
        except:
            pass


if __name__ == "__main__":
    main()

2025-09-10 11:10:28,781 - INFO - Starting Scenario Generation and Impact Prediction...
2025-09-10 11:10:28,782 - INFO - ============================================================
2025-09-10 11:10:28,782 - INFO - Step 1: Initializing H2O and loading model...


Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Europe/London
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,5 months and 13 days
H2O_cluster_name:,H2O_from_python_jxq370_rvkv9f
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,120 Gb
H2O_cluster_total_cores:,72
H2O_cluster_allowed_cores:,1
H2O_cluster_status:,"locked, healthy"


2025-09-10 11:10:28,802 - INFO - H2O initialized successfully
2025-09-10 11:10:29,137 - INFO - Loaded H2O model: StackedEnsemble_BestOfFamily_1_AutoML_4_20250903_145640
2025-09-10 11:10:29,187 - INFO - Loading feature importance from: /rds/homes/j/jxq370/shiz-wm-netzero/users/juncheng/EPC/automl/outputs/base_model_importance/combined_base_model_importance.csv
2025-09-10 11:10:29,189 - INFO - Loaded importance for 29 features
2025-09-10 11:10:29,190 - INFO - Top 10 most important features:
2025-09-10 11:10:29,190 - INFO -   WALLS_ENERGY_EFF: 0.99
2025-09-10 11:10:29,190 - INFO -   ROOF_ENERGY_EFF: 0.92
2025-09-10 11:10:29,191 - INFO -   CONSTRUCTION_AGE_BAND: 0.70
2025-09-10 11:10:29,191 - INFO -   MAINHEAT_ENERGY_EFF: 0.58
2025-09-10 11:10:29,191 - INFO -   HOT_WATER_ENERGY_EFF: 0.56
2025-09-10 11:10:29,191 - INFO -   MAIN_FUEL: 0.48
2025-09-10 11:10:29,192 - INFO -   WINDOWS_ENERGY_EFF: 0.41
2025-09-10 11:10:29,192 - INFO -   TOTAL_FLOOR_AREA: 0.33
2025-09-10 11:10:29,192 - INFO -   H

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


2025-09-10 11:50:34,983 - INFO - Scenario predictions completed
2025-09-10 11:50:34,983 - INFO - 
2025-09-10 11:50:34,984 - INFO - PREDICTION SUMMARY
2025-09-10 11:50:34,984 - INFO - ============================================================
2025-09-10 11:50:34,984 - INFO - 
A0 (Moderate) Scenario:
2025-09-10 11:50:34,988 - INFO -   Homes improved: 122571/122847 (99.8%)
2025-09-10 11:50:34,988 - INFO -   Achieved C or better: 121614/122847 (99.0%)
2025-09-10 11:50:34,989 - INFO -   Mean improvement: 1.52 bands
2025-09-10 11:50:34,989 - INFO - 
A1 (Ambitious) Scenario:
2025-09-10 11:50:34,993 - INFO -   Homes improved: 122847/122847 (100.0%)
2025-09-10 11:50:34,993 - INFO -   Achieved C or better: 122847/122847 (100.0%)
2025-09-10 11:50:34,994 - INFO -   Mean improvement: 1.53 bands
2025-09-10 11:50:35,084 - INFO - Step 6: Calculating costs and benefits...
2025-09-10 11:50:35,085 - INFO - Calculating costs and benefits...
2025-09-10 11:50:57,872 - INFO - 
2025-09-10 11:50:57,873 - INF

H2O session _sid_97a6 closed.

SCENARIO ANALYSIS COMPLETED SUCCESSFULLY!
Homes analyzed: 122847
A0 success rate: 99.0%
A1 success rate: 100.0%
A0 average cost: £28,200.09
A1 average cost: £29,426.87
Feature importance loaded: True
Rating band analysis completed: True
Results saved to: /rds/homes/j/jxq370/shiz-wm-netzero/users/juncheng/EPC/outputs/step_3_scenarios/scenarios_20250910_111028
